# CSE674 Project 1

## Libraries 

In [1]:
import pandas as pd
import numpy as np
import math
import time
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import K2Score
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.sampling import BayesianModelSampling
from  pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.inference import VariableElimination

# Task 1

### Loading CSV file data [CPDs and Marginal probabilities] into DataFrame

In [2]:
marg_probs = pd.read_csv('Table2.csv')
for i in range(1,7):
    marg_probs.iloc[:,i] = (marg_probs.iloc[:,i].str.split('%').str[0]).astype(float)/100
print(marg_probs)
cpd_x1 = pd.read_csv('Table3.csv')
for i in range(1,5):
    cpd_x1.iloc[:,i] = (cpd_x1.iloc[:,i].str.split('%').str[0]).astype(float)/100
print(cpd_x1)
cpd_x2 = pd.read_csv('Table4.csv')
for i in range(1,6):
    cpd_x2.iloc[:,i] = (cpd_x2.iloc[:,i].str.split('%').str[0]).astype(float)/100
print(cpd_x2)
cpd_x3 = pd.read_csv('Table5.csv')
for i in range(1,4):
    cpd_x3.iloc[:,i] = (cpd_x3.iloc[:,i].str.split('%').str[0]).astype(float)/100
print(cpd_x3)
cpd_x4 = pd.read_csv('Table6.csv')
for i in range(1,5):
    cpd_x4.iloc[:,i] = (cpd_x4.iloc[:,i].str.split('%').str[0]).astype(float)/100
print(cpd_x4)
cpd_x5 = pd.read_csv('Table7.csv')
for i in range(1,5):
    cpd_x5.iloc[:,i] = (cpd_x5.iloc[:,i].str.split('%').str[0]).astype(float)/100
print(cpd_x5)
cpd_x6 = pd.read_csv('Table8.csv')
for i in range(1,6):
    cpd_x6.iloc[:,i] = (cpd_x6.iloc[:,i].str.split('%').str[0]).astype(float)/100
print(cpd_x6)





   Values  x1 (Relative height to t to h)  x2 (Shape of h loop)  \
0  a = x0                           0.780                 0.275   
1  b = x1                           0.015                 0.320   
2  c = x2                           0.055                 0.025   
3  d = x3                           0.150                 0.170   
4  e = x4                             NaN                 0.210   

   x3 (Shape of h arch)  x4 (Height of Cross )  x5 (Baseline of h)  \
0                  0.18                  0.715               0.375   
1                  0.66                  0.105               0.110   
2                  0.16                  0.010               0.105   
3                   NaN                  0.170               0.410   
4                   NaN                    NaN                 NaN   

   x6 (Shape of t)  
0            0.015  
1            0.320  
2            0.140  
3            0.315  
4            0.210  
                        x1  x01 (t shorter than h)

### Calculating the correlations using Cross Entropy between x values

In [3]:
# correlation calculations for x1 and x2 [CROSS ENTROPY]
H12 = 0
for i in range(1,5): # column number of x1 CPD table
    for j in range(1,6): # row numbers that have CPDs related to x2 in x1 CPD table
        p_x1x2 = cpd_x1.iloc[j,i] * marg_probs.iloc[i-1,1]
        p_x1 = marg_probs.iloc[i-1,1]
        p_x2 = marg_probs.iloc[j-1,2]
        H12 += p_x1x2*math.log(p_x1*p_x2)
H12 = -H12
print("Cross Entropy with CPD X2 given X1 --- ", H12)

# correlation calculations for x1 and x4 [CROSS ENTROPY]
H14 = 0
for i in range(1,5): # column number of x1 CPD table
    for j in range(6,10): # row numbers that have CPDs related to x4 in x1 CPD table
        p_x1x4 = cpd_x1.iloc[j,i] * marg_probs.iloc[i-1,1]
        p_x1 = marg_probs.iloc[i-1,1]
        p_x4 = marg_probs.iloc[j-6,4]
        H14 += p_x1x4*math.log(p_x1*p_x4)
H14 = -H14
print("Cross Entropy with CPD X4 given X1 --- ", H14)

# correlation calculations for x1 and x6 [CROSS ENTROPY]
H16 = 0
for i in range(1,5): # column number of x1 CPD table
    for j in range(11,15): # row numbers that have CPDs related to x6 in x1 CPD table
        p_x1x6 = cpd_x1.iloc[j,i] * marg_probs.iloc[i-1,1]
        p_x1 = marg_probs.iloc[i-1,1]
        p_x6 = marg_probs.iloc[j-11,6]
        H16 += p_x1x6*math.log(p_x1*p_x6)
H16 = -H16
print("Cross Entropy with CPD X6 given X1 --- ", H16)

# correlation calculations for x2 and x3 [CROSS ENTROPY]
H23 = 0
for i in range(1,6): # column number of x2 CPD table
    for j in range(1,4): # row numbers that have CPDs related to x3 in x2 CPD table
        p_x2x3 = cpd_x2.iloc[j,i] * marg_probs.iloc[i-1,2]
        p_x2 = marg_probs.iloc[i-1,2]
        p_x3 = marg_probs.iloc[j-1,3]
        H23 += p_x2x3*math.log(p_x2*p_x3)
H23 = -H23
print("Cross Entropy with CPD X3 given X2 --- ", H23)

# correlation calculations for x2 and x5 [CROSS ENTROPY]
H25 = 0
for i in range(1,6): # column number of x2 CPD table
    for j in range(4,8): # row numbers that have CPDs related to x5 in x2 CPD table
        p_x2x5 = cpd_x2.iloc[j,i] * marg_probs.iloc[i-1,2]
        p_x2 = marg_probs.iloc[i-1,2]
        p_x5 = marg_probs.iloc[j-4,5]
        H25 += p_x2x5*math.log(p_x2*p_x5)
H25 = -H25
print("Cross Entropy with CPD X5 given X2 --- ", H25)

# correlation calculations for x3 and x2 [CROSS ENTROPY]
H32 = 0
for i in range(1,4): # column number of x3 CPD table
    for j in range(1,6): # row numbers that have CPDs related to x2 in x3 CPD table
        p_x3x2 = cpd_x3.iloc[j,i] * marg_probs.iloc[i-1,3]
        p_x3 = marg_probs.iloc[i-1,3]
        p_x2 = marg_probs.iloc[j-1,2]
        H32 += p_x3x2*math.log(p_x3*p_x2)
H32 = -H32
print("Cross Entropy with CPD X2 given X3 --- ", H32)

# correlation calculations for x3 and x5 [CROSS ENTROPY]
H35 = 0
for i in range(1,4): # column number of x3 CPD table
    for j in range(6,10): # row numbers that have CPDs related to x2 in x3 CPD table
        p_x3x5 = cpd_x3.iloc[j,i] * marg_probs.iloc[i-1,3]
        p_x3 = marg_probs.iloc[i-1,3]
        p_x5 = marg_probs.iloc[j-6,5]
        H35 += p_x3x5*math.log(p_x3*p_x5)
H35 = -H35
print("Cross Entropy with CPD X5 given X3 --- ", H35)

# correlation calculations for x3 and x6 [CROSS ENTROPY]
H36 = 0
for i in range(1,4): # column number of x3 CPD table
    for j in range(10,15): # row numbers that have CPDs related to x2 in x3 CPD table
        p_x3x6 = cpd_x3.iloc[j,i] * marg_probs.iloc[i-1,3]            
        p_x3 = marg_probs.iloc[i-1,3]
        p_x6 = marg_probs.iloc[j-10,6]
        H36 += p_x3x6*math.log(p_x3*p_x6)
H36 = -H36
print("Cross Entropy with CPD X6 given X3 --- ", H36)

# correlation calculations for x4 and x1 [CROSS ENTROPY]
H41 = 0
for i in range(1,5): # column number of x4 CPD table
    for j in range(1,5): # row numbers that have CPDs related to x1 in x4 CPD table
        p_x4x1 = cpd_x4.iloc[j,i] * marg_probs.iloc[i-1,4]
        p_x4 = marg_probs.iloc[i-1,4]
        p_x1 = marg_probs.iloc[j-1,1]
        H41 += p_x4x1*math.log(p_x4*p_x1)
H41 = -H41
print("Cross Entropy with CPD X1 given X4 --- ", H41)

# correlation calculations for x4 and x2 [CROSS ENTROPY]
H42 = 0
for i in range(1,5): # column number of x4 CPD table
    for j in range(5,10): # row numbers that have CPDs related to x2 in x4 CPD table
        p_x4x2 = cpd_x4.iloc[j,i] * marg_probs.iloc[i-1,4]
        p_x4 = marg_probs.iloc[i-1,4]
        p_x2 = marg_probs.iloc[j-5,2]
        H42 += p_x4x2*math.log(p_x4*p_x2)
H42 = -H42
print("Cross Entropy with CPD X2 given X4 --- ", H42)

# correlation calculations for x4 and x6 [CROSS ENTROPY]
H46 = 0
for i in range(1,5): # column number of x4 CPD table
    for j in range(10,15): # row numbers that have CPDs related to x6 in x4 CPD table
        p_x4x6 = cpd_x4.iloc[j,i] * marg_probs.iloc[i-1,4]
        p_x4 = marg_probs.iloc[i-1,4]
        p_x6 = marg_probs.iloc[j-10,6]
        H46 += p_x4x6*math.log(p_x4*p_x6)
H46 = -H46
print("Cross Entropy with CPD X6 given X4 --- ", H46)

# correlation calculations for x5 and x2 [CROSS ENTROPY]
H52 = 0
for i in range(1,4): # column number of x5 CPD table
    for j in range(1,6): # row numbers that have CPDs related to x2 in x5 CPD table
        p_x5x2 = cpd_x5.iloc[j,i] * marg_probs.iloc[i-1,5]
        p_x5 = marg_probs.iloc[i-1,5]
        p_x2 = marg_probs.iloc[j-1,2]
        H52 += p_x5x2*math.log(p_x5*p_x2)
H52 = -H52
print("Cross Entropy with CPD X2 given X5--- ", H52)

# correlation calculations for x5 and x3 [CROSS ENTROPY]
H53 = 0
for i in range(1,4): # column number of x5 CPD table
    for j in range(6,9): # row numbers that have CPDs related to x3 in x5 CPD table
        p_x5x3 = cpd_x5.iloc[j,i] * marg_probs.iloc[i-1,5]
        p_x5 = marg_probs.iloc[i-1,5]
        p_x3 = marg_probs.iloc[j-6,3]
        H53 += p_x5x3*math.log(p_x5*p_x3)
H53 = -H53
print("Cross Entropy with CPD X3 given X5 --- ", H53)

# correlation calculations for x6 and x1 [CROSS ENTROPY]
H61 = 0
for i in range(1,6): # column number of x6 CPD table
    for j in range(1,5): # row numbers that have CPDs related to x1 in x6 CPD table
        p_x6x1 = cpd_x6.iloc[j,i] * marg_probs.iloc[i-1,6] 
        p_x6 = marg_probs.iloc[i-1,6]
        p_x1 = marg_probs.iloc[j-1,1]
        H61 += p_x6x1*math.log(p_x6*p_x1)
H61 = -H61
print("Cross Entropy with CPD X1 given X6 --- ", H61)

# correlation calculations for x6 and x2 [CROSS ENTROPY]
H62 = 0
for i in range(1,6): # column number of x6 CPD table
    for j in range(5,10): # row numbers that have CPDs related to x1 in x6 CPD table
        p_x6x2 = cpd_x6.iloc[j,i] * marg_probs.iloc[i-1,6]
        p_x6 = marg_probs.iloc[i-1,6]
        p_x2 = marg_probs.iloc[j-5,2]
        H62 += p_x6x2*math.log(p_x6*p_x2)
H62 = -H62
print("Cross Entropy with CPD X2 given X6 --- ", H62)

# correlation calculations for x6 and x3 [CROSS ENTROPY]
H63 = 0
for i in range(1,6): # column number of x6 CPD table
    for j in range(10,13): # row numbers that have CPDs related to x1 in x6 CPD table
        p_x6x3 = cpd_x6.iloc[j,i] * marg_probs.iloc[i-1,6]
        p_x6 = marg_probs.iloc[i-1,6]
        p_x3 = marg_probs.iloc[j-10,3]
        H63 += p_x6x3*math.log(p_x6*p_x3)
H63 = -H63
print("Cross Entropy with CPD X3 given X6 --- ", H63)

# correlation calculations for x6 and x4 [CROSS ENTROPY]
H64 = 0
for i in range(1,6): # column number of x6 CPD table
    for j in range(13,17): # row numbers that have CPDs related to x1 in x6 CPD table
        p_x6x4 = cpd_x6.iloc[j,i] * marg_probs.iloc[i-1,6]
        p_x6 = marg_probs.iloc[i-1,6]
        p_x4 = marg_probs.iloc[j-13,4]
        H64 += p_x6x4*math.log(p_x6*p_x4)
H64 = -H64
print("Cross Entropy with CPD X4 given X6 --- ", H64)
cross_entropy = [H12, H14, H16, H23, H25, H32, H35, H36, H41, H42, H46, H52, H53, H61, H62, H63, H64]
cross_entropy.sort()

Cross Entropy with CPD X2 given X1 ---  2.1424370668269535
Cross Entropy with CPD X4 given X1 ---  1.525266106826817
Cross Entropy with CPD X6 given X1 ---  3.06100302852479
Cross Entropy with CPD X3 given X2 ---  2.316355718267753
Cross Entropy with CPD X5 given X2 ---  2.653441952398932
Cross Entropy with CPD X2 given X3 ---  2.3174823468319103
Cross Entropy with CPD X5 given X3 ---  2.089543895607478
Cross Entropy with CPD X6 given X3 ---  2.2711959851218246
Cross Entropy with CPD X1 given X4 ---  1.5251101836816672
Cross Entropy with CPD X2 given X4 ---  2.272021831298273
Cross Entropy with CPD X6 given X4 ---  2.2186394422986013
Cross Entropy with CPD X2 given X5---  1.722062405410663
Cross Entropy with CPD X3 given X5 ---  1.3532661612805736
Cross Entropy with CPD X1 given X6 ---  2.181545535526468
Cross Entropy with CPD X2 given X6 ---  2.830851716653726
Cross Entropy with CPD X3 given X6 ---  2.284671704533646
Cross Entropy with CPD X4 given X6 ---  2.217914493026952


# Task 2

### Given CPD data Pre-processing 

In [4]:
# Putting marginal probabilities into seperate individual lists
x1_mp = []
x2_mp = []
x3_mp = []
x4_mp = []
x5_mp = []
x6_mp = []
for i in range(4):
    x1_mp.append(marg_probs.iloc[i,1])
for i in range(5):
    x2_mp.append(marg_probs.iloc[i,2])
for i in range(3):
    x3_mp.append(marg_probs.iloc[i,3])
for i in range(4):
    x4_mp.append(marg_probs.iloc[i,4])
for i in range(4):
    x5_mp.append(marg_probs.iloc[i,5])
for i in range(5):
    x6_mp.append(marg_probs.iloc[i,6])

# Putting conditional probabilities into seperate individual lists
cpd_givenX1 = []
cpd_givenX2 = []
cpd_givenX3 = []
cpd_givenX4 = []
cpd_givenX5 = []
cpd_givenX6 = []

for i in range(1,5): # columns of cpd X1
    for j in  range(1,15): # all rows of cpd X1
        cpd_givenX1.append(cpd_x1.iloc[j,i])
for i in range(1,6): # columns of cpd X2
    for j in range(1,8): # all rows of cpd X2
        cpd_givenX2.append(cpd_x2.iloc[j,i])
for i in range(1,4): # columns of cpd X3
    for j in range(1,15): # all rows of cpd X3
        cpd_givenX3.append(cpd_x3.iloc[j,i])
for i in range(1,5): # columns of cpd X4
    for j in range(1,15): # all rows of cpd X4
        cpd_givenX4.append(cpd_x4.iloc[j,i])
for i in range(1,5): # columns of cpd X5
    for j in range(1,9): # all rows of cpd X5
        cpd_givenX5.append(cpd_x5.iloc[j,i])
for i in range(1,6): # columns of cpd X6
    for j in range(1,17): # all rows of cpd X6
        cpd_givenX6.append(cpd_x6.iloc[j,i])



### Creating dataset for Bayesian Model 1, creating the Bayesian Model 1 and Getting its K2 score

In [5]:
#put all marginal probabilities and conditional probabilties into lists seperately. 
#The conditional prob pair values from several columns and rows come in one list.
# create 10 BN manually
#use numpy random choice to sample values from each of these lists based on BN. hard this -  from which list to sample
#loop the above step for 1000 times
# do the above 2 steps 10 times for 10 diffferent BN.
# we have 10 datasets with 1000 data points each


# Getting Dataset for BN: 1 --- get 1000 data points by sampling 
# p(x5).p(x1|x4).p(x4|x6).p(x6|x3).p(x3|x5)p(x2|x5)

dataset_BN1 = pd.DataFrame(columns=['x1','x2','x3','x4','x5','x6'])
cpd_X1X4 = []
cpd_X4X6 = []
cpd_X6X3 = []
cpd_X3X5 = []
cpd_X2X5 = []

cpd_X1X4.append(cpd_givenX4[0:4])
cpd_X1X4.append(cpd_givenX4[14:18])
cpd_X1X4.append(cpd_givenX4[28:32])
cpd_X1X4.append(cpd_givenX4[42:46])

cpd_X4X6.append(cpd_givenX6[12:16])
cpd_X4X6.append(cpd_givenX6[28:32])
cpd_X4X6.append(cpd_givenX6[44:48])
cpd_X4X6.append(cpd_givenX6[60:64])
cpd_X4X6.append(cpd_givenX6[76:80])

cpd_X6X3.append(cpd_givenX3[9:14])
cpd_X6X3.append(cpd_givenX3[23:28])
cpd_X6X3.append(cpd_givenX3[37:42])

cpd_X3X5.append(cpd_givenX5[5:8])
cpd_X3X5.append(cpd_givenX5[13:16])
cpd_X3X5.append(cpd_givenX5[21:24])
cpd_X3X5.append(cpd_givenX5[29:32])

cpd_X2X5.append(cpd_givenX5[0:5])
cpd_X2X5.append(cpd_givenX5[8:13])
cpd_X2X5.append(cpd_givenX5[16:21])
cpd_X2X5.append(cpd_givenX5[24:29])

#cpd_X1X4[:] = [float(x) / 4 for x in cpd_X1X4]
#cpd_X4X6[:] = [float(x) / 5 for x in cpd_X4X6]
#cpd_X6X3[:] = [float(x) / 3 for x in cpd_X6X3]
#cpd_X3X5[:] = [float(x) / 4 for x in cpd_X3X5]
#cpd_X2X5[:] = [float(x) / 4 for x in cpd_X2X5]

features_x1 = [0,1,2,3]
features_x2 = [0,1,2,3,4]
features_x3 = [0,1,2]
features_x4 = [0,1,2,3]
features_x5 = [0,1,2,3]
features_x6 = [0,1,2,3,4]

for i in range(1000):
    x5 = np.random.choice(features_x5, p = x5_mp)
    #print(p1)
    x2 = np.random.choice(features_x2, p = cpd_X2X5[x5])
    #print(p2)
    x3 = np.random.choice(features_x3, p = cpd_X3X5[x5])
    #print(p3)
    x6 = np.random.choice(features_x6, p = cpd_X6X3[x3])
    #print(p4)
    x4 = np.random.choice(features_x4, p = cpd_X4X6[x6])
    #print(p5)
    x1 = np.random.choice(features_x1, p = cpd_X1X4[x4])
    #print(p6)
    dataset_BN1 = dataset_BN1.append({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'x6': x6}, ignore_index=True)
print(dataset_BN1)
BN_model1 =  BayesianModel([('x4', 'x1'),('x6','x4'),('x3','x6'),('x5','x3'),('x5','x2')])   
k2Score = K2Score(dataset_BN1)
score1 = k2Score.score(BN_model1)

    x1 x2 x3 x4 x5 x6
0    2  3  1  0  0  4
1    0  1  0  0  2  2
2    0  0  0  1  0  3
3    0  1  1  0  3  2
4    0  1  1  0  0  2
5    0  4  1  0  3  3
6    0  1  1  0  3  3
7    0  0  1  3  3  4
8    3  2  1  0  1  4
9    0  0  1  0  3  1
10   0  3  0  0  3  2
11   0  4  1  1  3  4
12   0  3  2  0  3  3
13   3  2  1  1  0  1
14   0  2  0  3  1  1
15   0  1  1  0  3  4
16   0  0  1  0  0  4
17   2  1  1  0  0  3
18   3  4  1  1  1  1
19   0  0  0  0  0  4
20   0  3  1  0  1  3
21   3  3  1  0  3  1
22   0  3  1  0  0  2
23   3  1  1  0  3  3
24   0  1  0  3  0  3
25   0  0  1  0  1  2
26   2  1  1  1  0  2
27   0  0  1  0  0  4
28   0  4  1  3  0  4
29   0  4  1  0  3  1
..  .. .. .. .. .. ..
970  2  4  1  0  0  2
971  0  0  0  3  2  3
972  0  3  1  0  3  1
973  0  4  1  0  3  1
974  0  1  1  3  0  3
975  0  4  1  0  1  1
976  0  3  1  0  3  4
977  0  0  1  0  3  4
978  0  3  0  0  2  1
979  0  1  1  0  3  0
980  0  1  2  0  1  3
981  0  1  1  1  1  2
982  0  1  1  0  3  3
983  3  3 

### Creating dataset for Bayesian Model 2, creating the Bayesian Model 2 and Getting its K2 score

In [6]:
# Getting Dataset for BN: 2 --- get 1000 data points by sampling 
# p(x1).p(x4|x1).p(x6|x4).p(x2|x6).p(x3|x2)p(x5|x2)

dataset_BN2 = pd.DataFrame(columns=['x1','x2','x3','x4','x5','x6'])
cpd_X4X1 = []
cpd_X6X4 = []
cpd_X2X6 = []
cpd_X3X2 = []
cpd_X5X2 = []

#print(cpd_givenX1)
cpd_X4X1.append(cpd_givenX1[5:9])
cpd_X4X1.append(cpd_givenX1[19:23])
cpd_X4X1.append(cpd_givenX1[33:37])
cpd_X4X1.append(cpd_givenX1[47:51])

cpd_X6X4.append(cpd_givenX4[9:14])
cpd_X6X4.append(cpd_givenX4[23:28])
cpd_X6X4.append(cpd_givenX4[37:42])
cpd_X6X4.append(cpd_givenX4[51:56])
cpd_X6X4[0][3]=round(cpd_X6X4[0][3],3)
cpd_X6X4[1][1]=round(cpd_X6X4[1][1],3)
cpd_X6X4[1][2]=round(cpd_X6X4[1][2],3)
cpd_X6X4[3][2]=round(cpd_X6X4[3][2],3)
cpd_X6X4[3][3]=round(cpd_X6X4[3][3],3)
cpd_X6X4[1][1] = 0.572
cpd_X6X4[0][3] = 0.328
cpd_X6X4[3][3]= 0.324

cpd_X2X6.append(cpd_givenX6[4:9])
cpd_X2X6.append(cpd_givenX6[20:25])
cpd_X2X6.append(cpd_givenX6[36:41])
cpd_X2X6.append(cpd_givenX6[52:57])
cpd_X2X6.append(cpd_givenX6[68:73])
cpd_X2X6[0][1] = round(cpd_X2X6[0][1],3)
cpd_X2X6[0][1] = 0.334
cpd_X2X6[0][3] = round(cpd_X2X6[0][3],3)
cpd_X2X6[1][1] = round(cpd_X2X6[1][1],3)
cpd_X2X6[1][1] = 0.298
cpd_X2X6[1][3] = round(cpd_X2X6[1][3],3)
cpd_X2X6[4][0] = round(cpd_X2X6[4][0],3)
cpd_X2X6[4][3] = round(cpd_X2X6[4][4],3)
cpd_X2X6[2][3] = 0.144
cpd_X2X6[3][0] = 0.319
cpd_X2X6[4][3] = 0.167

cpd_X3X2.append(cpd_givenX2[0:3])
cpd_X3X2.append(cpd_givenX2[7:10])
cpd_X3X2.append(cpd_givenX2[14:17])
cpd_X3X2.append(cpd_givenX2[21:24])
cpd_X3X2.append(cpd_givenX2[28:31])
cpd_X3X2[0][0] = 0.128
cpd_X3X2[1][1] = round(cpd_X3X2[1][1],3)
cpd_X3X2[3][0] = round(cpd_X3X2[3][0],3)
cpd_X3X2[3][2] = round(cpd_X3X2[3][2],3)
cpd_X3X2[4][0] = round(cpd_X3X2[4][0],3)

cpd_X5X2.append(cpd_givenX2[3:7])
cpd_X5X2.append(cpd_givenX2[10:14])
cpd_X5X2.append(cpd_givenX2[17:21])
cpd_X5X2.append(cpd_givenX2[24:28])
cpd_X5X2.append(cpd_givenX2[31:35])

#cpd_X1X4[:] = [float(x) / 4 for x in cpd_X1X4]
#cpd_X4X6[:] = [float(x) / 5 for x in cpd_X4X6]
#cpd_X6X3[:] = [float(x) / 3 for x in cpd_X6X3]
#cpd_X3X5[:] = [float(x) / 4 for x in cpd_X3X5]
#cpd_X2X5[:] = [float(x) / 4 for x in cpd_X2X5]

features_x1 = [0,1,2,3]
features_x2 = [0,1,2,3,4]
features_x3 = [0,1,2]
features_x4 = [0,1,2,3]
features_x5 = [0,1,2,3]
features_x6 = [0,1,2,3,4]

for i in range(1000):
    x1 = np.random.choice(features_x1, p = x1_mp)
    x4 = np.random.choice(features_x4, p = cpd_X4X1[x1])
    x6 = np.random.choice(features_x6, p = cpd_X6X4[x4])
    x2 = np.random.choice(features_x2, p = cpd_X2X6[x6])
    x3 = np.random.choice(features_x3, p = cpd_X3X2[x2])
    x5 = np.random.choice(features_x5, p = cpd_X5X2[x2])
    dataset_BN2 = dataset_BN2.append({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'x6': x6}, ignore_index=True)
print(dataset_BN2)
BN_model2 =  BayesianModel([('x1', 'x4'),('x4','x6'),('x6','x2'),('x2','x3'),('x2','x5')])   
k2Score2 = K2Score(dataset_BN2)
score2 = k2Score2.score(BN_model2)

    x1 x2 x3 x4 x5 x6
0    0  0  1  0  0  3
1    1  2  1  0  0  3
2    0  3  1  0  1  4
3    0  1  1  0  1  3
4    0  1  1  1  0  2
5    3  0  1  1  0  2
6    0  0  0  0  0  1
7    0  4  0  0  3  2
8    0  0  2  0  0  1
9    0  3  1  0  3  3
10   0  0  1  0  0  1
11   0  0  1  0  0  3
12   0  4  1  0  3  2
13   0  2  1  0  0  3
14   0  0  0  1  3  1
15   0  0  2  0  0  1
16   0  0  1  0  0  1
17   0  1  1  0  2  3
18   0  1  1  3  3  3
19   0  3  1  0  2  3
20   0  0  1  0  0  3
21   0  3  1  0  0  1
22   0  1  1  0  2  2
23   3  4  1  0  3  3
24   0  0  1  0  2  1
25   0  1  1  0  0  3
26   0  1  1  0  3  1
27   0  4  1  0  0  4
28   2  1  0  0  0  4
29   0  1  1  0  1  2
..  .. .. .. .. .. ..
970  0  1  1  0  0  1
971  3  0  1  3  0  2
972  2  0  1  0  3  4
973  0  4  1  0  1  2
974  2  1  1  0  0  4
975  3  3  1  1  1  1
976  0  1  1  0  0  2
977  0  3  1  0  3  3
978  0  1  1  0  2  1
979  0  4  2  0  0  4
980  0  0  1  0  3  3
981  0  1  0  0  3  4
982  0  1  1  0  1  3
983  2  0 

### Creating dataset for Bayesian Model 3, creating the Bayesian Model 3 and Getting its K2 score

In [7]:
# Getting Dataset for BN: 3 --- get 1000 data points by sampling 
# p(x5).p(x1|x6).p(x3|x5).p(x6|x3).p(x4|x6)p(x2|x4)

dataset_BN3 = pd.DataFrame(columns=['x1','x2','x3','x4','x5','x6'])
cpd_X1X6 = []
cpd_X3X5 = []
cpd_X6X3 = []
cpd_X4X6 = []
cpd_X2X4 = []

cpd_X1X6.append(cpd_givenX6[0:4])
cpd_X1X6.append(cpd_givenX6[16:20])
cpd_X1X6.append(cpd_givenX6[32:36])
cpd_X1X6.append(cpd_givenX6[48:52])
cpd_X1X6.append(cpd_givenX6[64:68])
cpd_X1X6[1][2] = round(cpd_X1X6[1][2],3)
cpd_X1X6[1][3] = round(cpd_X1X6[1][3],3)
cpd_X1X6[2][0] = round(cpd_X1X6[2][0],3)
cpd_X1X6[2][1] = round(cpd_X1X6[2][1],3)
cpd_X1X6[2][2] = round(cpd_X1X6[2][2],3)
cpd_X1X6[3][3] = round(cpd_X1X6[3][3],3)
cpd_X1X6[4][3] = round(cpd_X1X6[4][3],3)
cpd_X1X6[1][1] =  0.027
cpd_X1X6[3][3] =  0.127

cpd_X3X5.append(cpd_givenX5[5:8])
cpd_X3X5.append(cpd_givenX5[13:16])
cpd_X3X5.append(cpd_givenX5[21:24])
cpd_X3X5.append(cpd_givenX5[29:32])

cpd_X6X3.append(cpd_givenX3[9:14])
cpd_X6X3.append(cpd_givenX3[23:28])
cpd_X6X3.append(cpd_givenX3[37:42])

cpd_X4X6.append(cpd_givenX6[12:16])
cpd_X4X6.append(cpd_givenX6[28:32])
cpd_X4X6.append(cpd_givenX6[44:48])
cpd_X4X6.append(cpd_givenX6[60:64])
cpd_X4X6.append(cpd_givenX6[76:80])

cpd_X2X4.append(cpd_givenX4[4:9])
cpd_X2X4.append(cpd_givenX4[18:23])
cpd_X2X4.append(cpd_givenX4[32:37])
cpd_X2X4.append(cpd_givenX4[46:51])
cpd_X2X4[0][2] = round(cpd_X2X4[0][2],3)
cpd_X2X4[1][0] = round(cpd_X2X4[1][0],3)
cpd_X2X4[1][1] = round(cpd_X2X4[1][1],3)
cpd_X2X4[1][4] = round(cpd_X2X4[1][4],3)
cpd_X2X4[3][0] = round(cpd_X2X4[3][0],3)
cpd_X2X4[3][1] = round(cpd_X2X4[3][1],3)
cpd_X2X4[3][2] = round(cpd_X2X4[3][1],3)
cpd_X2X4[0][2] = 0.02
cpd_X2X4[3][2] = 0.029
cpd_X2X4[3][0] = 0.178

#cpd_X1X4[:] = [float(x) / 4 for x in cpd_X1X4]
#cpd_X4X6[:] = [float(x) / 5 for x in cpd_X4X6]
#cpd_X6X3[:] = [float(x) / 3 for x in cpd_X6X3]
#cpd_X3X5[:] = [float(x) / 4 for x in cpd_X3X5]
#cpd_X2X5[:] = [float(x) / 4 for x in cpd_X2X5]

features_x1 = [0,1,2,3]
features_x2 = [0,1,2,3,4]
features_x3 = [0,1,2]
features_x4 = [0,1,2,3]
features_x5 = [0,1,2,3]
features_x6 = [0,1,2,3,4]

for i in range(1000):
    x5 = np.random.choice(features_x5, p = x5_mp)
    x1 = np.random.choice(features_x1, p = cpd_X1X6[x5])
    x3 = np.random.choice(features_x3, p = cpd_X3X5[x5])
    x6 = np.random.choice(features_x6, p = cpd_X6X3[x3])
    x4 = np.random.choice(features_x4, p = cpd_X4X6[x6])
    x2 = np.random.choice(features_x2, p = cpd_X2X4[x4])
    dataset_BN3 = dataset_BN3.append({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'x6': x6}, ignore_index=True)
print(dataset_BN3)
BN_model3 =  BayesianModel([('x5', 'x3'),('x3','x6'),('x6','x4'),('x4','x2'),('x6','x1')])   
k2Score3 = K2Score(dataset_BN3)
score3 = k2Score3.score(BN_model3)

    x1 x2 x3 x4 x5 x6
0    0  0  1  0  3  4
1    0  4  1  0  3  3
2    3  0  0  0  2  3
3    0  4  2  0  0  1
4    0  3  1  3  0  3
5    0  4  0  0  0  4
6    0  0  1  0  3  1
7    0  1  0  0  3  1
8    0  1  2  0  3  1
9    0  4  1  0  3  3
10   0  1  1  3  0  3
11   0  1  1  1  0  1
12   0  3  0  0  2  2
13   0  4  1  1  0  1
14   0  1  1  0  3  3
15   0  1  1  0  3  3
16   0  4  1  0  3  1
17   0  0  2  0  3  3
18   0  3  1  0  0  3
19   0  3  1  3  1  3
20   0  4  2  0  2  3
21   3  1  1  3  2  3
22   0  4  1  1  3  1
23   0  1  1  0  0  2
24   0  0  1  3  3  1
25   0  0  0  0  2  1
26   0  1  0  0  2  4
27   3  1  2  0  1  2
28   0  0  1  0  3  2
29   3  1  1  0  3  1
..  .. .. .. .. .. ..
970  0  1  0  3  0  3
971  3  1  2  0  1  3
972  2  4  1  0  2  3
973  0  0  1  0  3  4
974  0  4  1  1  0  2
975  0  3  1  1  2  2
976  0  4  2  0  3  3
977  0  3  1  3  0  1
978  2  3  1  1  1  2
979  0  3  2  0  1  3
980  0  3  1  3  3  3
981  0  1  1  3  3  4
982  0  1  0  0  3  1
983  0  3 

### Creating dataset for Bayesian Model 4, creating the Bayesian Model 4 and Getting its K2 score

In [8]:
# Getting Dataset for BN: 4 --- get 1000 data points by sampling 
# p(x1).p(x4|x3).p(x2|x3).p(x3|x6).p(x6|x1)p(x5|x1)

dataset_BN4 = pd.DataFrame(columns=['x1','x2','x3','x4','x5','x6'])
cpd_X4X1 = []
cpd_X2X3 = []
cpd_X3X6 = []
cpd_X6X1 = []
cpd_X5X3 = []

cpd_X4X1.append(cpd_givenX1[5:9])
cpd_X4X1.append(cpd_givenX1[19:23])
cpd_X4X1.append(cpd_givenX1[33:37])
cpd_X4X1.append(cpd_givenX1[47:51])

cpd_X2X3.append(cpd_givenX3[0:5])
cpd_X2X3.append(cpd_givenX3[14:19])
cpd_X2X3.append(cpd_givenX3[28:33])
for i in range(3):
    for j in range(5):
        cpd_X2X3[i][j] =  round(cpd_X2X3[i][j],3)

cpd_X3X6.append(cpd_givenX6[9:12])
cpd_X3X6.append(cpd_givenX6[25:28])
cpd_X3X6.append(cpd_givenX6[41:44])
cpd_X3X6.append(cpd_givenX6[57:60])
cpd_X3X6.append(cpd_givenX6[73:76])
cpd_X3X6[0][0] =  round(cpd_X3X6[0][0],3)
cpd_X3X6[2][1] =  round(cpd_X3X6[2][1],3)
cpd_X3X6[2][2] = round(cpd_X3X6[2][2],3)
cpd_X3X6[3][1] = round(cpd_X3X6[3][1],3)
cpd_X3X6[4][0] =  round(cpd_X3X6[4][0],3)
cpd_X3X6[0][0] = 0.174
cpd_X3X6[2][2] = 0.048
cpd_X3X6[3][1] = 0.647

cpd_X6X1.append(cpd_givenX1[9:14])
cpd_X6X1.append(cpd_givenX1[23:28])
cpd_X6X1.append(cpd_givenX1[37:42])
cpd_X6X1.append(cpd_givenX1[51:56])
cpd_X6X1[0][3] = round(cpd_X6X1[0][3], 3)
cpd_X6X1[1][1] = round(cpd_X6X1[1][1], 3)
cpd_X6X1[1][2] = round(cpd_X6X1[1][2], 3)
cpd_X6X1[0][4] = 0.219
cpd_X6X1[1][2] = 0.334

cpd_X5X3.append(cpd_givenX3[5:9])
cpd_X5X3.append(cpd_givenX3[19:23])
cpd_X5X3.append(cpd_givenX3[33:37])
for i in range(3):
    for j in range(4):
        cpd_X5X3[i][j] =  round(cpd_X5X3[i][j],3)
cpd_X5X3[0][3] = 0.334
cpd_X5X3[1][3] = 0.401

#cpd_X1X4[:] = [float(x) / 4 for x in cpd_X1X4]
#cpd_X4X6[:] = [float(x) / 5 for x in cpd_X4X6]
#cpd_X6X3[:] = [float(x) / 3 for x in cpd_X6X3]
#cpd_X3X5[:] = [float(x) / 4 for x in cpd_X3X5]
#cpd_X2X5[:] = [float(x) / 4 for x in cpd_X2X5]

features_x1 = [0,1,2,3]
features_x2 = [0,1,2,3,4]
features_x3 = [0,1,2]
features_x4 = [0,1,2,3]
features_x5 = [0,1,2,3]
features_x6 = [0,1,2,3,4]

for i in range(1000):
    x1 = np.random.choice(features_x1, p = x1_mp)
    x4 = np.random.choice(features_x4, p = cpd_X4X1[x1])
    x6 = np.random.choice(features_x6, p = cpd_X6X1[x1])
    x3 = np.random.choice(features_x3, p = cpd_X3X6[x6])
    x2 = np.random.choice(features_x2, p = cpd_X2X3[x3])
    x5 = np.random.choice(features_x5, p = cpd_X5X3[x3])
    dataset_BN4 = dataset_BN4.append({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'x6': x6}, ignore_index=True)
print(dataset_BN4)
BN_model4 =  BayesianModel([('x1', 'x4'),('x1','x6'),('x3','x2'),('x6','x3'),('x3','x5')])   
k2Score4 = K2Score(dataset_BN4)
score4 = k2Score4.score(BN_model4)

    x1 x2 x3 x4 x5 x6
0    3  1  2  0  3  3
1    0  0  1  0  3  1
2    0  4  1  0  1  1
3    0  3  2  0  0  4
4    0  4  1  0  0  3
5    0  1  1  0  3  3
6    1  0  1  0  3  1
7    0  0  1  0  0  1
8    3  2  0  0  0  2
9    0  0  1  0  0  3
10   0  1  1  0  0  4
11   2  0  1  0  3  4
12   0  3  1  3  2  4
13   0  3  2  3  3  3
14   3  1  1  3  3  4
15   0  4  0  0  0  2
16   0  3  1  0  1  3
17   3  0  1  1  1  2
18   0  4  0  0  0  4
19   3  4  1  3  1  1
20   0  2  1  3  1  1
21   0  4  0  1  0  1
22   0  4  1  0  0  1
23   0  3  0  0  0  4
24   0  1  1  0  3  3
25   0  3  1  1  0  3
26   3  0  1  0  0  2
27   0  0  1  0  0  1
28   0  4  1  0  2  4
29   0  0  1  0  0  4
..  .. .. .. .. .. ..
970  0  3  1  0  0  3
971  0  0  1  0  2  3
972  0  4  1  0  3  3
973  0  3  1  0  0  4
974  0  0  1  0  3  1
975  0  3  1  0  2  3
976  0  3  1  0  0  2
977  0  0  1  3  3  3
978  0  0  1  3  3  1
979  0  3  1  0  0  1
980  0  3  1  0  3  2
981  0  0  1  0  3  1
982  3  3  0  0  3  2
983  0  0 

### Creating dataset for Bayesian Model 5, creating the Bayesian Model 5 and Getting its K2 score

In [9]:
# Getting Dataset for BN: 5 --- get 1000 data points by sampling 
# p(x1).p(x6|x1).p(x2|x6).p(x5|x2).p(x3|x2)p(x4|x6)

dataset_BN5 = pd.DataFrame(columns=['x1','x2','x3','x4','x5','x6'])
cpd_X6X1 = []
cpd_X2X6 = []
cpd_X5X2 = []
cpd_X3X2 = []
cpd_X4X6 = []

cpd_X6X1.append(cpd_givenX1[9:14])
cpd_X6X1.append(cpd_givenX1[23:28])
cpd_X6X1.append(cpd_givenX1[37:42])
cpd_X6X1.append(cpd_givenX1[51:56])
cpd_X6X1[0][3] = round(cpd_X6X1[0][3], 3)
cpd_X6X1[1][1] = round(cpd_X6X1[1][1], 3)
cpd_X6X1[1][2] = round(cpd_X6X1[1][2], 3)
cpd_X6X1[0][4] = 0.219
cpd_X6X1[1][2] = 0.334

cpd_X2X6.append(cpd_givenX6[4:9])
cpd_X2X6.append(cpd_givenX6[20:25])
cpd_X2X6.append(cpd_givenX6[36:41])
cpd_X2X6.append(cpd_givenX6[52:57])
cpd_X2X6.append(cpd_givenX6[68:73])
cpd_X2X6[0][1] = round(cpd_X2X6[0][1],3)
cpd_X2X6[0][1] = 0.334
cpd_X2X6[0][3] = round(cpd_X2X6[0][3],3)
cpd_X2X6[1][1] = round(cpd_X2X6[1][1],3)
cpd_X2X6[1][1] = 0.298
cpd_X2X6[1][3] = round(cpd_X2X6[1][3],3)
cpd_X2X6[4][0] = round(cpd_X2X6[4][0],3)
cpd_X2X6[4][3] = round(cpd_X2X6[4][4],3)
cpd_X2X6[2][3] = 0.144
cpd_X2X6[3][0] = 0.319
cpd_X2X6[4][3] = 0.167

cpd_X5X2.append(cpd_givenX2[3:7])
cpd_X5X2.append(cpd_givenX2[10:14])
cpd_X5X2.append(cpd_givenX2[17:21])
cpd_X5X2.append(cpd_givenX2[24:28])
cpd_X5X2.append(cpd_givenX2[31:35])

cpd_X3X2.append(cpd_givenX2[0:3])
cpd_X3X2.append(cpd_givenX2[7:10])
cpd_X3X2.append(cpd_givenX2[14:17])
cpd_X3X2.append(cpd_givenX2[21:24])
cpd_X3X2.append(cpd_givenX2[28:31])
cpd_X3X2[0][0] = 0.128
cpd_X3X2[1][1] = round(cpd_X3X2[1][1],3)
cpd_X3X2[3][0] = round(cpd_X3X2[3][0],3)
cpd_X3X2[3][2] = round(cpd_X3X2[3][2],3)
cpd_X3X2[4][0] = round(cpd_X3X2[4][0],3)

cpd_X4X6.append(cpd_givenX6[12:16])
cpd_X4X6.append(cpd_givenX6[28:32])
cpd_X4X6.append(cpd_givenX6[44:48])
cpd_X4X6.append(cpd_givenX6[60:64])
cpd_X4X6.append(cpd_givenX6[76:80])

#cpd_X1X4[:] = [float(x) / 4 for x in cpd_X1X4]
#cpd_X4X6[:] = [float(x) / 5 for x in cpd_X4X6]
#cpd_X6X3[:] = [float(x) / 3 for x in cpd_X6X3]
#cpd_X3X5[:] = [float(x) / 4 for x in cpd_X3X5]
#cpd_X2X5[:] = [float(x) / 4 for x in cpd_X2X5]

features_x1 = [0,1,2,3]
features_x2 = [0,1,2,3,4]
features_x3 = [0,1,2]
features_x4 = [0,1,2,3]
features_x5 = [0,1,2,3]
features_x6 = [0,1,2,3,4]

for i in range(1000):
    x1 = np.random.choice(features_x1, p = x1_mp)
    x6 = np.random.choice(features_x6, p = cpd_X6X1[x1])
    x2 = np.random.choice(features_x2, p = cpd_X2X6[x6])
    x5 = np.random.choice(features_x5, p = cpd_X5X2[x2])
    x3 = np.random.choice(features_x3, p = cpd_X3X2[x2])
    x4 = np.random.choice(features_x4, p = cpd_X4X6[x6])
    dataset_BN5 = dataset_BN5.append({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'x6': x6}, ignore_index=True)
print(dataset_BN5)
BN_model5 =  BayesianModel([('x1', 'x6'),('x6','x2'),('x2','x5'),('x2','x3'),('x6','x4')])   
k2Score5 = K2Score(dataset_BN5)
score5 = k2Score5.score(BN_model5)

    x1 x2 x3 x4 x5 x6
0    0  0  1  0  3  3
1    3  1  1  0  0  4
2    0  1  0  0  0  3
3    3  1  1  0  0  4
4    0  1  0  0  2  3
5    0  1  0  0  1  4
6    0  4  1  0  0  4
7    0  0  1  3  2  1
8    0  4  2  0  0  1
9    0  0  1  1  3  1
10   1  0  1  0  0  2
11   3  0  1  0  2  2
12   3  0  1  0  0  4
13   0  0  1  0  1  4
14   0  1  1  3  0  3
15   3  1  0  0  3  3
16   0  2  1  0  0  3
17   0  0  0  3  3  3
18   0  1  0  0  3  4
19   0  4  1  0  0  4
20   0  0  1  0  0  1
21   2  0  1  0  3  1
22   0  3  1  3  3  3
23   0  0  1  1  0  2
24   3  4  1  3  0  4
25   1  0  1  3  3  1
26   0  3  1  0  1  3
27   0  4  2  0  0  1
28   0  3  1  1  0  1
29   0  0  0  0  0  4
..  .. .. .. .. .. ..
970  3  0  1  0  2  3
971  0  1  1  0  3  3
972  0  1  1  0  0  1
973  0  1  1  3  0  1
974  1  0  1  0  0  1
975  0  2  1  0  0  3
976  0  0  0  1  0  1
977  3  0  1  3  3  1
978  0  4  0  0  2  1
979  0  0  2  3  0  3
980  0  1  1  0  3  1
981  0  0  1  0  0  1
982  0  2  1  0  1  3
983  0  1 

### Finding the Best Model based on K2 Score

In [10]:
K2score_df = pd.DataFrame(columns=['BN 1','BN 2','BN 3','BN 4','BN 5'])
K2score_df = K2score_df.append({'BN 1':score1, 'BN 2':score2, 'BN 3':score3, 'BN 4':score4, 'BN 5':score5},ignore_index=True)
print(K2score_df)

          BN 1         BN 2         BN 3         BN 4         BN 5
0 -6445.690859 -6492.836637 -6195.182877 -6417.558673 -6468.877617


### Finding the Highest Probabable "th" and Least Probabable "th"

In [11]:
joint_prob = []
for i in range(1000):
    p5 = x5_mp[dataset_BN3.iloc[i,4]]
    p3 = cpd_X3X5[dataset_BN3.iloc[i,4]][dataset_BN3.iloc[i,2]]
    p6 = cpd_X6X3[dataset_BN3.iloc[i,2]][dataset_BN3.iloc[i,5]]
    p1 = cpd_X1X6[dataset_BN3.iloc[i,5]][dataset_BN3.iloc[i,0]]
    p4 = cpd_X4X6[dataset_BN3.iloc[i,5]][dataset_BN3.iloc[i,3]]
    p2 = cpd_X2X4[dataset_BN3.iloc[i,3]][dataset_BN3.iloc[i,1]]
    jprob = p1*p2*p3*p4*p5*p6
    joint_prob.append(jprob)
print(joint_prob)
max_th = np.argmax(joint_prob)
jpmax = np.max(joint_prob)
print(jpmax)
print(max_th)
print(dataset_BN3.iloc[max_th])
min_th = np.argmin(joint_prob)
print(min_th)
print(dataset_BN3.iloc[min_th])


[0.00995116448275488, 0.01025982394328808, 0.00042141961070747995, 0.0010510177702500003, 0.00283108899346875, 0.001267811821017, 0.01117370095790256, 0.0032245807409205605, 0.00293819863617, 0.01025982394328808, 0.0038912414676187507, 0.003040760638773, 0.00028696824766763994, 0.003040760638773, 0.01685542504968756, 0.01685542504968756, 0.007110536973210721, 0.00532846757580984, 0.007908737457159, 0.0008160933575024999, 0.00017199928275012, 0.00013200915708224995, 0.00309941431180524, 0.007148434721904, 0.00192900611014452, 0.0020613211667017198, 0.00128437398700254, 0.00016915345102655997, 0.006969525311105279, 0.0024315404559045596, 0.006332559216298559, 0.0020590514658845997, 0.00057420889596, 0.0026943250179195005, 0.00099498432089616, 0.0033427874188170005, 0.016122580482309842, 0.0026358794880300004, 0.0007618915870706252, 0.00018245167590935998, 0.0, 0.0003455220800459999, 0.0004758161479047901, 0.0031313458270710005, 0.0011840367085551, 0.011460532580658003, 0.0030994143118052

### Inference from the Best Bayesian Model  - Bayesian Network 3

In [31]:
# Inference
cpd_givenX1 = []
cpd_givenX2 = []
cpd_givenX3 = []
cpd_givenX4 = []
cpd_givenX5 = []
cpd_givenX6 = []

for i in range(1,15): # columns of cpd X1
    for j in  range(1,5): # all rows of cpd X1
        cpd_givenX1.append(cpd_x1.iloc[i,j])
for i in range(1,8): # columns of cpd X2
    for j in range(1,6): # all rows of cpd X2
        cpd_givenX2.append(cpd_x2.iloc[i,j])
for i in range(1,15): # columns of cpd X3
    for j in range(1,4): # all rows of cpd X3
        cpd_givenX3.append(cpd_x3.iloc[i,j])
for i in range(1,15): # columns of cpd X4
    for j in range(1,5): # all rows of cpd X4
        cpd_givenX4.append(cpd_x4.iloc[i,j])
for i in range(1,9): # columns of cpd X5
    for j in range(1,5): # all rows of cpd X5
        cpd_givenX5.append(cpd_x5.iloc[i,j])
for i in range(1,17): # columns of cpd X6
    for j in range(1,6): # all rows of cpd X6
        cpd_givenX6.append(cpd_x6.iloc[i,j])

cpd_X1X6 = []
cpd_X3X5 = []
cpd_X6X3 = []
cpd_X4X6 = []
cpd_X2X4 = []

cpd_X1X6.append(cpd_givenX6[0:5])
cpd_X1X6.append(cpd_givenX6[5:10])
cpd_X1X6.append(cpd_givenX6[10:15])
cpd_X1X6.append(cpd_givenX6[15:20])
cpd_X1X6[2][1] = 0.142
cpd_X1X6[3][1] = 0.140
cpd_X1X6[0][2] = 0.714
cpd_X1X6[1][2] = 0.036
cpd_X1X6[2][2] = 0.036
cpd_X1X6[0][3] = 0.794
cpd_X1X6[3][3] = 0.206
cpd_X1X6[3][4] = 0.143
cpd_X3X5.append(cpd_givenX5[20:24])
cpd_X3X5.append(cpd_givenX5[24:28])
cpd_X3X5.append(cpd_givenX5[28:32])

cpd_X6X3.append(cpd_givenX3[27:30])
cpd_X6X3.append(cpd_givenX3[30:33])
cpd_X6X3.append(cpd_givenX3[33:36])
cpd_X6X3.append(cpd_givenX3[36:39])
cpd_X6X3.append(cpd_givenX3[39:42])

cpd_X4X6.append(cpd_givenX6[60:65])
cpd_X4X6.append(cpd_givenX6[65:70])
cpd_X4X6.append(cpd_givenX6[70:75])
cpd_X4X6.append(cpd_givenX6[75:80])

cpd_X2X4.append(cpd_givenX4[16:20])
cpd_X2X4.append(cpd_givenX4[20:24])
cpd_X2X4.append(cpd_givenX4[24:28])
cpd_X2X4.append(cpd_givenX4[28:32])
cpd_X2X4.append(cpd_givenX4[32:36])

cpdx3 = TabularCPD('x3', 3, cpd_X3X5, evidence=['x5'], evidence_card=[4])
print(cpdx3)
cpdx6 = TabularCPD('x6', 5, cpd_X6X3, evidence=['x3'], evidence_card=[3])
print(cpdx6)
cpdx1 = TabularCPD('x1', 4, cpd_X1X6, evidence=['x6'], evidence_card=[5])
print(cpdx1)
cpdx4 = TabularCPD('x4', 4, cpd_X4X6, evidence=['x6'], evidence_card=[5])
print(cpdx2)
cpdx5 = TabularCPD('x5',4,[x5_mp])
print(cpdx5)
cpdx2 = TabularCPD('x2',5, cpd_X2X4, evidence=['x4'], evidence_card=[4])
print(cpdx2)

BN_model3.add_cpds(cpdx2)
BN_model3.add_cpds(cpdx3)
BN_model3.add_cpds(cpdx4)
BN_model3.add_cpds(cpdx5)
BN_model3.add_cpds(cpdx6)
BN_model3.add_cpds(cpdx1)

startBN = time.time()
infer = VariableElimination(BN_model3)
print(infer.query(['x1']) ['x1'])
print(infer.query(['x3'], evidence={'x2': 0}) ['x3'])
print(infer.query(['x4'], evidence={'x3': 0, 'x2':4 }) ['x4'])
print(infer.query(['x5'], evidence={'x1': 1}) ['x5'])
print(time.time()-startBN)

+------+---------------------+-------+----------------------+-------+
| x5   | x5_0                | x5_1  | x5_2                 | x5_3  |
+------+---------------------+-------+----------------------+-------+
| x3_0 | 0.17300000000000001 | 0.136 | 0.381                | 0.146 |
+------+---------------------+-------+----------------------+-------+
| x3_1 | 0.6940000000000001  | 0.682 | 0.578                | 0.647 |
+------+---------------------+-------+----------------------+-------+
| x3_2 | 0.133               | 0.182 | 0.040999999999999995 | 0.207 |
+------+---------------------+-------+----------------------+-------+
+------+---------------------+-------+-------+
| x3   | x3_0                | x3_1  | x3_2  |
+------+---------------------+-------+-------+
| x6_0 | 0.0                 | 0.023 | 0.0   |
+------+---------------------+-------+-------+
| x6_1 | 0.389               | 0.318 | 0.25  |
+------+---------------------+-------+-------+
| x6_2 | 0.083               | 0.152 | 0.

# Task 3

### Convert the best Bayesian Network to Markov Model

In [29]:
mm = BN_model3.to_markov_model()
print(mm.nodes())
print(mm.edges())

['x5', 'x3', 'x6', 'x4', 'x1', 'x2']
[('x5', 'x3'), ('x3', 'x6'), ('x6', 'x4'), ('x6', 'x1'), ('x4', 'x2')]


### Infering from the Markov Model

In [32]:
startMN = time.time()
infer1 = VariableElimination(mm)
print(infer1.query(['x1']) ['x1'])
print(infer1.query(['x3'], evidence={'x2': 0}) ['x3'])
print(infer1.query(['x4'], evidence={'x3': 0, 'x2':4 }) ['x4'])
print(infer1.query(['x5'], evidence={'x1': 1}) ['x5'])
print(time.time()-startMN)

+------+-----------+
| x1   |   phi(x1) |
+======+===========+
| x1_0 |    0.7549 |
+------+-----------+
| x1_1 |    0.0150 |
+------+-----------+
| x1_2 |    0.0605 |
+------+-----------+
| x1_3 |    0.1697 |
+------+-----------+
+------+-----------+
| x3   |   phi(x3) |
+======+===========+
| x3_0 |    0.1788 |
+------+-----------+
| x3_1 |    0.6622 |
+------+-----------+
| x3_2 |    0.1590 |
+------+-----------+
+------+-----------+
| x4   |   phi(x4) |
+======+===========+
| x4_0 |    0.6512 |
+------+-----------+
| x4_1 |    0.1523 |
+------+-----------+
| x4_2 |    0.0000 |
+------+-----------+
| x4_3 |    0.1965 |
+------+-----------+
+------+-----------+
| x5   |   phi(x5) |
+======+===========+
| x5_0 |    0.3763 |
+------+-----------+
| x5_1 |    0.1098 |
+------+-----------+
| x5_2 |    0.1062 |
+------+-----------+
| x5_3 |    0.4077 |
+------+-----------+
0.023990392684936523


# Task 4

### Converting the CSV file of "AND" dataset into a python dataframe

In [15]:
and_df = pd.read_csv('AND-Features(1).csv').drop(['Unnamed: 0'],axis=1)
del and_df['img_id']
print(and_df)

      f1  f2  f3  f4  f5  f6  f7  f8  f9
0      2   1   1   0   2   2   0   2   2
1      2   1   1   0   2   2   0   1   2
2      2   1   1   3   2   2   0   2   2
3      1   1   1   0   2   2   0   2   2
4      2   1   1   0   2   2   0   0   2
5      2   1   1   3   2   2   0   1   2
6      2   1   1   0   2   2   0   3   2
7      1   1   1   0   2   2   0   1   2
8      1   1   1   3   2   2   0   2   2
9      2   1   1   3   2   2   0   0   2
10     3   1   1   0   2   2   0   2   2
11     2   1   1   3   2   2   0   3   2
12     1   1   1   0   2   2   0   0   2
13     2   1   1   0   2   2   1   2   2
14     1   1   1   3   2   2   0   1   2
15     1   1   1   0   2   2   0   3   2
16     3   1   1   0   2   2   0   1   2
17     3   1   1   3   2   2   0   2   2
18     2   1   1   0   2   2   1   1   2
19     1   1   1   3   2   2   0   0   2
20     2   1   1   3   2   2   1   2   2
21     2   1   1   1   2   2   0   2   2
22     1   1   1   3   2   2   0   3   2
23     3   1   1

### Using hill climb search to find the best model using the scoring method of k2 score

In [16]:
k2 = K2Score(and_df)
hc = HillClimbSearch(and_df, scoring_method=k2)
best_model = hc.estimate()
print("--- Best Model ---")
print(best_model.edges())
k2Score_best = k2.score(best_model)
print("---K2 Score of Best Model---")
print(k2Score_best)

--- Best Model ---
[('f3', 'f4'), ('f3', 'f9'), ('f3', 'f8'), ('f5', 'f9'), ('f5', 'f3'), ('f9', 'f8'), ('f9', 'f7'), ('f9', 'f1'), ('f9', 'f6'), ('f9', 'f2'), ('f9', 'f4')]
---K2 Score of Best Model---
-9462.704892371388


### Creating the Best Bayesian Model and getting CPDs 

In [17]:
model = BayesianModel(best_model.edges())
model.fit(and_df)
cpds = model.get_cpds()
print(cpds)

[<TabularCPD representing P(f1:4 | f9:2) at 0x1d418e4da20>, <TabularCPD representing P(f2:5 | f9:2) at 0x1d418f4deb8>, <TabularCPD representing P(f3:3 | f5:4) at 0x1d418f4f390>, <TabularCPD representing P(f4:5 | f3:3, f9:2) at 0x1d418f44470>, <TabularCPD representing P(f5:4) at 0x1d418e5e518>, <TabularCPD representing P(f6:4 | f9:2) at 0x1d418f44be0>, <TabularCPD representing P(f7:4 | f9:2) at 0x1d418e44978>, <TabularCPD representing P(f8:5 | f3:3, f9:2) at 0x1d418f53048>, <TabularCPD representing P(f9:2 | f3:3, f5:4) at 0x1d418e4d5f8>]


### Infering from BN 

In [18]:
startBN1 = time.time()
infer1 = VariableElimination(model)
print(infer1.query(['f1']) ['f1'])
print(infer1.query(['f3'], evidence={'f2': 0}) ['f3'])
print(infer1.query(['f4'], evidence={'f3': 0, 'f2':1, 'f5':0}) ['f4'])
print(infer1.query(['f5'], evidence={'f1': 1}) ['f5'])
print(time.time()-startBN1)

+------+-----------+
| f1   |   phi(f1) |
+======+===========+
| f1_0 |    0.1267 |
+------+-----------+
| f1_1 |    0.3021 |
+------+-----------+
| f1_2 |    0.3460 |
+------+-----------+
| f1_3 |    0.2251 |
+------+-----------+
+------+-----------+
| f3   |   phi(f3) |
+======+===========+
| f3_0 |    0.1294 |
+------+-----------+
| f3_1 |    0.8341 |
+------+-----------+
| f3_2 |    0.0366 |
+------+-----------+
+------+-----------+
| f4   |   phi(f4) |
+======+===========+
| f4_0 |    0.1337 |
+------+-----------+
| f4_1 |    0.4544 |
+------+-----------+
| f4_2 |    0.3109 |
+------+-----------+
| f4_3 |    0.0928 |
+------+-----------+
| f4_4 |    0.0081 |
+------+-----------+
+------+-----------+
| f5   |   phi(f5) |
+======+===========+
| f5_0 |    0.2134 |
+------+-----------+
| f5_1 |    0.0325 |
+------+-----------+
| f5_2 |    0.7413 |
+------+-----------+
| f5_3 |    0.0127 |
+------+-----------+
0.04404115676879883


### Converting the AND Dataset BN to Markov Model

In [19]:
mm1 = best_model.to_markov_model()
print(mm1.nodes())
print(mm1.edges())

['f1', 'f9', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8']
[('f1', 'f9'), ('f9', 'f2'), ('f9', 'f3'), ('f9', 'f4'), ('f9', 'f5'), ('f9', 'f6'), ('f9', 'f7'), ('f9', 'f8'), ('f3', 'f4'), ('f3', 'f8'), ('f3', 'f5')]
